### Imports and Environment

In [18]:
import gymnasium as gym
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

import numpy as np

import collections
import gymnasium as gym
import numpy as np
import os

import tqdm

from matplotlib import pyplot as plt
from typing import Any, List, Sequence, Tuple

import pickle

import path_datatype
import sys

from env import tradingEng

# Define environment
file_path = os.path.abspath("../MarketGeneratingFunctions/1.6kRunDemo.pkl")
with open(file_path,"rb") as fp:
    paths = pickle.load(fp)
env = tradingEng(paths)

In [20]:
from stable_baselines3.common.env_checker import check_env

check_env(env, warn=True, skip_render_check=True)

{'Swaption Position': array([nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'Q Position': array([nan, nan, nan, nan, nan, nan, nan, nan, nan])}
<bound method tradingEng._get_obs of <env.tradingEng object at 0x24260fd10>>
{'Swaption Position': array([nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'Q Position': array([nan, nan, nan, nan, nan, nan, nan, nan, nan])}
<bound method tradingEng._get_obs of <env.tradingEng object at 0x24260fd10>>


AssertionError: The observation returned by the `reset()` method does not match the lower bound of the given observation space Box([  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0. -inf], [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
 inf inf inf inf inf inf inf inf inf  1.  1.  1.  1.  1.  1.  1.  1.  1.
 inf], (37,), float64).Expected: obs >= -inf, actual min value: nan at index 0

### Making the Agent

In [11]:
n_actions = 18
action_noise = OrnsteinUhlenbeckActionNoise(mean = np.zeros(n_actions), sigma = 0.05*np.ones(n_actions), theta = 0.01)
model = DDPG("MlpPolicy", env, action_noise=action_noise, verbose=1, batch_size=25)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/isabellebyman/opt/anaconda3/envs/exjobb/lib/python3.13/site-packages/stable_baselines3/common/buffers.py:229: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 0.74GB > 0.32GB
  warnings.warn(


### Training the Agent

In [12]:
Nruns = 10
model.learn(total_timesteps=251*10*Nruns, log_interval=2)
model.save("ddpg_fin")

{'Swaption Position': array([nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'Q Position': array([nan, nan, nan, nan, nan, nan, nan, nan, nan])}
<bound method tradingEng._get_obs of <env.tradingEng object at 0x25ebaa780>>
{'Swaption Position': array([nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'Q Position': array([nan, nan, nan, nan, nan, nan, nan, nan, nan])}
<bound method tradingEng._get_obs of <env.tradingEng object at 0x25ebaa780>>
{'Swaption Position': array([nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'Q Position': array([nan, nan, nan, nan, nan, nan, nan, nan, nan])}
<bound method tradingEng._get_obs of <env.tradingEng object at 0x25ebaa780>>


RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float

### Run a test

In [ ]:

from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo
num_eval_episodes = 1

env = tradingEng(paths)

episode_over = False
rewards = list()
actions = list()
obs, info = env.reset()
while not episode_over:
    action, _states = model.predict(obs, deterministic=True)  # replace with actual agent
    obs, reward, terminated, truncated, info = env.step(action)
    rewards.append(reward)
    actions.append(action)
    episode_over = terminated or truncated
env.close()

print(f'Example action taken: {actions[0]}')
print(f'Episode rewards: {rewards}')

ValueError: You have passed a tuple to the predict() function instead of a Numpy array or a Dict. You are probably mixing Gym API with SB3 VecEnv API: `obs, info = env.reset()` (Gym) vs `obs = vec_env.reset()` (SB3 VecEnv). See related issue https://github.com/DLR-RM/stable-baselines3/issues/1694 and documentation for more information: https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecenv-api-vs-gym-api